In [1]:
import pickle
import numpy as np
import pandas as pd
import re
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    ffn_hyperparameter_search,
)

In [4]:
output_dir = "talklife_moc_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

Talklife MoC

In [5]:
%run load_talklifemoc.py

In [6]:
%run load_sbert-embeddings.py

In [7]:
sbert_embeddings.shape

torch.Size([18604, 384])

Baseline: FFN

In [8]:
num_epochs = 100
hidden_dim_sizes = [[64,64],[128,128],[256,256],[512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [9]:
#create indices for kfold
fold_col_names = [c for c in df.columns if 'fold' in c ]
fold_list = []
for foldc in fold_col_names:
    fold_list.append((df[df[foldc]=='train'].index, df[df[foldc]=='dev'].index, df[df[foldc]=='test'].index))
fold_list = tuple(fold_list)

In [10]:
ffn_current, best_ffn_current, _, __ = ffn_hyperparameter_search( 
    num_epochs=num_epochs,
    x_data=sbert_embeddings,
    y_data=y_data,
    hidden_dim_sizes=hidden_dim_sizes,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_ids=None, 
    split_indices=fold_list,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output= f"{output_dir}/ffn_current_focal_{gamma}.csv",
    verbose=False
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in talklife_moc_output/ffn_current_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in talklife_moc_output/ffn_current_focal_2_best_model.csv


In [11]:
best_ffn_current

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,valid_recall_scores,hidden_dim,dropout_rate,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,None,0.811976,0.534763,"[0.8914839985960882, 0.4354008335376317, 0.277...",0.532724,"[0.8933938735051481, 0.43044110518662143, 0.27...",0.536867,"[0.8895822720326032, 0.44047619047619047, 0.28...",None,0.805552,...,"[0.882609341720739, 0.454336147352264, 0.27006...","(256, 256)",0.2,0.0001,1,focal,2,True,5,64
0,None,0.809073,0.535849,"[0.8894868345185469, 0.43799472295514513, 0.28...",0.530379,"[0.894933608353745, 0.42405945192754296, 0.272...",0.541815,"[0.8841059602649006, 0.45287698412698413, 0.28...",None,0.804927,...,"[0.878101936698202, 0.47889485801995396, 0.280...","(256, 256)",0.2,0.0001,12,focal,2,True,5,64
0,None,0.806278,0.533401,"[0.8875971481790738, 0.4386336866902238, 0.273...",0.526237,"[0.8953608915381625, 0.417676087931808, 0.2656...",0.541526,"[0.8799668874172185, 0.4618055555555556, 0.282...",None,0.799633,...,"[0.8720590420526029, 0.4800460475825019, 0.273...","(256, 256)",0.2,0.0001,123,focal,2,True,5,64


In [20]:
best_ffn_current[['f1', 'f1_scores', 'precision', 
       'recall', 'valid_f1',
       'valid_f1_scores', 'valid_precision', 
       'valid_recall', 
       'hidden_dim', 'dropout_rate', 'learning_rate', 'seed',
       'loss_function', 'k_fold', 
        'batch_size']]

,f1,f1_scores,precision,recall,valid_f1,valid_f1_scores,valid_precision,valid_recall,hidden_dim,dropout_rate,learning_rate,seed,loss_function,k_fold,batch_size
0,0.534763,"[0.8914839985960882, 0.4354008335376317, 0.277...",0.532724,0.536867,0.531947,"[0.8871353181320323, 0.4334614680578437, 0.275...",0.528918,0.535671,"(256, 256)",0.2,0.0001,1,focal,True,64
0,0.535849,"[0.8894868345185469, 0.43799472295514513, 0.28...",0.530379,0.541815,0.537895,"[0.8861563070155707, 0.44731182795698926, 0.28...",0.531444,0.545699,"(256, 256)",0.2,0.0001,12,focal,True,64
0,0.533401,"[0.8875971481790738, 0.4386336866902238, 0.273...",0.526237,0.541526,0.531445,"[0.8828381597091639, 0.4395643007730147, 0.271...",0.523246,0.541839,"(256, 256)",0.2,0.0001,123,focal,True,64


In [12]:
best_ffn_current["f1"].mean()

0.5346711829749976

In [13]:
best_ffn_current["precision"].mean()

0.5297801399703711

In [14]:
best_ffn_current["recall"].mean()

0.5400693116249538

In [15]:
np.stack(best_ffn_current["f1_scores"]).mean(axis=0)

array([0.88952266, 0.43734308, 0.27714781])

In [16]:
np.stack(best_ffn_current["precision_scores"]).mean(axis=0)

array([0.89456279, 0.42405888, 0.27071875])

In [17]:
np.stack(best_ffn_current["recall_scores"]).mean(axis=0)

array([0.88455171, 0.45171958, 0.28393665])